In [1]:
import sys
from functools import reduce
import numpy as np
import cirq
from stabilizer_states import StabilizerStates
from stabilizer_toolkit.decompositions import rank2, validate_decompositions
from stabilizer_toolkit.magic_states import enumerate_ccz
from stabilizer_toolkit.helpers.unitary import get_tensored_unitary

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
np.set_printoptions(precision=3, linewidth=sys.maxsize, edgeitems=4, threshold=1024, suppress=True) 

For a three qubit $\mathtt{CCZ}$ magic state, usually denoted $|\mathtt{CCZ}\rangle$, we search for a decomposition within the dataset of three qubit stabilizer states. There are $1,080$ complex-valued states in this dataset.

In [4]:
S3 = StabilizerStates(3)
print(len(S3))

1080


In [24]:
CCZ, _ = next(enumerate_ccz(3))

We use a brute-force method of search for finding rank-2 decompositions of $|\mathtt{CCZ}\rangle$ by attempting to pair each stabilizer state with every other stabilizer state. There are $582,660$ such pairings resulting in $15$ decompositions, all of which are real-valued stabilizer states.

In [23]:
decompositions, coeffs = rank2.search_all_stabilizer_states(CCZ, S3)

2023-04-20 22:11:44,084	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|████████████████████████████████████████████████████████████████████████| 582660/582660 [00:32<00:00, 18159.08it/s]


In [21]:
validate_decompositions(CCZ, decompositions, coeffs)

15 decompositions
|ψ〉	= [ 0.354  0.354  0.354  0.354  0.354  0.354  0.354 -0.354]

✅	= [0.707] * [0.  0.5 0.  0.5 0.5 0.  0.5 0. ]
	+ [0.707] * [ 0.5  0.   0.5  0.   0.   0.5  0.  -0.5]

✅	= [0.707] * [0.5 0.  0.  0.5 0.  0.5 0.5 0. ]
	+ [0.707] * [ 0.   0.5  0.5  0.   0.5  0.   0.  -0.5]

✅	= [0.707] * [0.  0.5 0.5 0.  0.  0.5 0.5 0. ]
	+ [0.707] * [ 0.5  0.   0.   0.5  0.5  0.   0.  -0.5]

✅	= [0.707] * [0.5 0.  0.5 0.  0.5 0.  0.5 0. ]
	+ [0.707] * [ 0.   0.5  0.   0.5  0.   0.5  0.  -0.5]

✅	= [0.707] * [0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
	+ [0.707] * [ 0.5  0.5  0.   0.   0.   0.   0.5 -0.5]

✅	= [0.707] * [0.5 0.5 0.  0.  0.5 0.5 0.  0. ]
	+ [0.707] * [ 0.   0.   0.5  0.5  0.   0.   0.5 -0.5]

✅	= [0.707] * [0.5 0.5 0.5 0.5 0.  0.  0.  0. ]
	+ [0.707] * [ 0.   0.   0.   0.   0.5  0.5  0.5 -0.5]

✅	= [1.] * [ 0.354  0.354  0.354 -0.354  0.354  0.354  0.354 -0.354]
	+ [0.707] * [0. 0. 0. 1. 0. 0. 0. 0.]

✅	= [1.] * [ 0.354  0.354  0.354  0.354  0.354 -0.354  0.354 -0.354]
	+ [0.707] 

True

Since we know all decompositions are real decompositions, then we can optimize the brute force search by using only the real stabilizer states, which reduces the dataset down to $240$ states.

In [13]:
S3_real = StabilizerStates(3, 'real')
print(len(S3_real))

240


Performing a brute-force search over real stabilizer state we only have $28,680$ pairings, which is an order of magnitude faster to search through. We still find the $15$ decompositions we found before.

In [14]:
decompositions, coeffs = rank2.search_all_stabilizer_states(CCZ, S3_real)

2023-04-19 22:49:56,229	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|██████████████████████████████████████████████████████████████████████████| 28680/28680 [00:02<00:00, 12411.13it/s]


In [20]:
validate_decompositions(CCZ, decompositions, coeffs)

15 decompositions
|ψ〉	= [ 0.354  0.354  0.354  0.354  0.354  0.354  0.354 -0.354]

✅	= [0.707] * [0.  0.5 0.  0.5 0.5 0.  0.5 0. ]
	+ [0.707] * [ 0.5  0.   0.5  0.   0.   0.5  0.  -0.5]

✅	= [0.707] * [0.5 0.  0.  0.5 0.  0.5 0.5 0. ]
	+ [0.707] * [ 0.   0.5  0.5  0.   0.5  0.   0.  -0.5]

✅	= [0.707] * [0.  0.5 0.5 0.  0.  0.5 0.5 0. ]
	+ [0.707] * [ 0.5  0.   0.   0.5  0.5  0.   0.  -0.5]

✅	= [0.707] * [0.5 0.  0.5 0.  0.5 0.  0.5 0. ]
	+ [0.707] * [ 0.   0.5  0.   0.5  0.   0.5  0.  -0.5]

✅	= [0.707] * [0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
	+ [0.707] * [ 0.5  0.5  0.   0.   0.   0.   0.5 -0.5]

✅	= [0.707] * [0.5 0.5 0.  0.  0.5 0.5 0.  0. ]
	+ [0.707] * [ 0.   0.   0.5  0.5  0.   0.   0.5 -0.5]

✅	= [0.707] * [0.5 0.5 0.5 0.5 0.  0.  0.  0. ]
	+ [0.707] * [ 0.   0.   0.   0.   0.5  0.5  0.5 -0.5]

✅	= [1.] * [ 0.354  0.354  0.354 -0.354  0.354  0.354  0.354 -0.354]
	+ [0.707] * [0. 0. 0. 1. 0. 0. 0. 0.]

✅	= [1.] * [ 0.354  0.354  0.354  0.354  0.354 -0.354  0.354 -0.354]
	+ [0.707] 

True

Finally, we can use the optimized ternary search method from manuscript, which makes the search almost instantaneous.

In [17]:
decompositions, coeffs = rank2.ternary_search(CCZ, S3_real, debug=False)

2023-04-19 22:50:33,421	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
100%|███████████████████████████████████████████████████████████████████████████████| 240/240 [00:00<00:00, 2802.06it/s]


In [19]:
validate_decompositions(CCZ, decompositions, coeffs)

15 decompositions
|ψ〉	= [ 0.354  0.354  0.354  0.354  0.354  0.354  0.354 -0.354]

✅	= [0.707] * [0.  0.5 0.  0.5 0.5 0.  0.5 0. ]
	+ [0.707] * [ 0.5  0.   0.5  0.   0.   0.5  0.  -0.5]

✅	= [0.707] * [0.5 0.  0.  0.5 0.  0.5 0.5 0. ]
	+ [0.707] * [ 0.   0.5  0.5  0.   0.5  0.   0.  -0.5]

✅	= [0.707] * [0.  0.5 0.5 0.  0.  0.5 0.5 0. ]
	+ [0.707] * [ 0.5  0.   0.   0.5  0.5  0.   0.  -0.5]

✅	= [0.707] * [0.5 0.  0.5 0.  0.5 0.  0.5 0. ]
	+ [0.707] * [ 0.   0.5  0.   0.5  0.   0.5  0.  -0.5]

✅	= [0.707] * [0.  0.  0.5 0.5 0.5 0.5 0.  0. ]
	+ [0.707] * [ 0.5  0.5  0.   0.   0.   0.   0.5 -0.5]

✅	= [0.707] * [0.5 0.5 0.  0.  0.5 0.5 0.  0. ]
	+ [0.707] * [ 0.   0.   0.5  0.5  0.   0.   0.5 -0.5]

✅	= [0.707] * [0.5 0.5 0.5 0.5 0.  0.  0.  0. ]
	+ [0.707] * [ 0.   0.   0.   0.   0.5  0.5  0.5 -0.5]

✅	= [1.] * [ 0.354  0.354  0.354 -0.354  0.354  0.354  0.354 -0.354]
	+ [0.707] * [0. 0. 0. 1. 0. 0. 0. 0.]

✅	= [1.] * [ 0.354  0.354  0.354  0.354  0.354 -0.354  0.354 -0.354]
	+ [0.707] 

True